In [1]:
import numpy as np
from numpy import linalg as LA
from ncon import ncon
import matplotlib.pyplot as plt

In [2]:
# creating TT (tensor train)

def TT(d, n, r):
    TT_left = np.random.rand(n,r)
    TT_right = np.random.rand(r,n)

    TT_cores = []
    TT_cores.append(TT_left)
    for i in range(d-2):
        TT_mid = np.random.rand(r,n,r)
        TT_cores.append(TT_mid)
    TT_cores.append(TT_right)

    TT_indices = []
    TT_indices.append([-1,1])
    for i in range(d-2):
        TT_indices.append([(i+1),-(i+2),(i+2)])
    TT_indices.append([(d-1),-d])

    TensorTrain = ncon(TT_cores, TT_indices)
    return TensorTrain

In [3]:
d = 3
n = 2
r = 5
A = TT(d,n,r)
B = TT(d,n,r)

In [71]:
d = 10
n = 2
r = 5

A_TT_left = np.random.rand(n,r)
A_TT_right = np.random.rand(r,n)
A_TT_cores = []
A_TT_cores.append(A_TT_left)
for i in range(d-2):
    A_TT_mid = np.random.rand(r,n,r)
    A_TT_cores.append(A_TT_mid)
A_TT_cores.append(A_TT_right)
A_TT_indices = []
A_TT_indices.append([-1,1])
for i in range(d-2):
    A_TT_indices.append([(i+1),-(i+2),(i+2)])
A_TT_indices.append([(d-1),-d])
A = ncon(A_TT_cores, A_TT_indices)

B_TT_left = np.random.rand(n,r)
B_TT_right = np.random.rand(r,n)
B_TT_cores = []
B_TT_cores.append(B_TT_left)
for i in range(d-2):
    B_TT_mid = np.random.rand(r,n,r)
    B_TT_cores.append(B_TT_mid)
B_TT_cores.append(B_TT_right)
B_TT_indices = []
B_TT_indices.append([-1,1])
for i in range(d-2):
    B_TT_indices.append([(i+1),-(i+2),(i+2)])
B_TT_indices.append([(d-1),-d])
B = ncon(B_TT_cores, B_TT_indices)

In [72]:
# scalar product of 2 TT

C_left = np.einsum('ij,ik->jk', A_TT_cores[0], B_TT_cores[0])
for i in range(d-2):
    C_left = np.einsum('ij,jkl->ikl', C_left, A_TT_cores[i+1])
    C_left = np.einsum('kji,ijl->kl', C_left, B_TT_cores[i+1])
C_left = np.einsum('ij,jk->ik', C_left, A_TT_cores[d-1])
result = np.einsum('ji,ji', C_left, B_TT_cores[d-1])

In [73]:
result

2423640797.579336

In [74]:
# unit vector

d = 10
n = 2
r = 1

Unit_vector = []
Unit_vector.append([1,0])
E = np.array([1,0])
for i in range(1,d):
    if (i+1 == 2):
        Unit_vector.append([0,1])
        E = np.tensordot(np.array([0,1]), E, axes=0)
    else:
        Unit_vector.append([1,0])
        E = np.tensordot(np.array([1,0]), E, axes=0)
Unit_vector = np.array(Unit_vector)
        
E.reshape(n**d)

array([0, 0, 1, ..., 0, 0, 0])

In [75]:
A.reshape(n**d)

array([1234.85666003, 1657.1494953 , 1090.63037435, ..., 1237.49065299,
        831.23144607, 1403.13506641])

In [76]:
Unit_vector

array([[1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

In [82]:
A_t = []
A_t.append(np.einsum('j,jk->k', Unit_vector[0], A_TT_cores[0]))
for i in range(d-2):
    A_t.append(np.einsum('j,kjl->kl', Unit_vector[i+1], A_TT_cores[i+1]))
A_t.append(np.einsum('j,kj->k', Unit_vector[d-1], A_TT_cores[d-1]))

res = np.einsum('j,jk->k', A_t[0], A_t[1])
for i in range(2,d-2):
    res = np.einsum('j,jk->k', res, A_t[i])
res = np.einsum('j,j', res, A_t[d-1])
res

478.0898442392939